In [9]:
! pip install cohere


  Using cached cohere-5.15.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached fastavro-1.11.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.7 kB)
  Using cached types_requests-2.32.4.20250611-py3-none-any.whl.metadata (2.1 kB)
Using cached cohere-5.15.0-py3-none-any.whl (259 kB)
Using cached fastavro-1.11.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
Using cached types_requests-2.32.4.20250611-py3-none-any.whl (20 kB)


In [7]:
import os
import voyageai
from dotenv import load_dotenv
load_dotenv()
from langchain_pinecone import PineconeEmbeddings
import cohere
from pinecone import Pinecone
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
#get the file path from local disk
import os
file_path=r"/home/spencer/Downloads/SpencerNyaenyaResume (9).pdf"
if os.path.exists(file_path):
    print("File exists")
else:
    print("file does not exist")

File exists


In [1]:
# loading data from an xml file
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

def extract_from_xml_url(url):
    try:
        response=requests.get(url) #gets the url
        response.raise_for_status # raise for any exceptions
        print(f"response :{response}")
        soup=BeautifulSoup(response.content,'xml') #for easier extraction
        print(f"soup : {soup}")

        #extract all texts
        all_texts=[]
        text=soup.get_text(separator=' ',strip=True)
        all_texts.append(text)
        print(f"all texts :{all_texts}")

        return all_texts

    except Exception as e:
        print (f"error processing the xml file {e}")
        return []  

In [2]:
url='https://www.w3schools.com/xml/plant_catalog.xml'
extract_from_xml_url(url)

response :<Response [200]>
soup : <?xml version="1.0" encoding="utf-8"?>
<CATALOG>
<PLANT>
<COMMON>Bloodroot</COMMON>
<BOTANICAL>Sanguinaria canadensis</BOTANICAL>
<ZONE>4</ZONE>
<LIGHT>Mostly Shady</LIGHT>
<PRICE>$2.44</PRICE>
<AVAILABILITY>031599</AVAILABILITY>
</PLANT>
<PLANT>
<COMMON>Columbine</COMMON>
<BOTANICAL>Aquilegia canadensis</BOTANICAL>
<ZONE>3</ZONE>
<LIGHT>Mostly Shady</LIGHT>
<PRICE>$9.37</PRICE>
<AVAILABILITY>030699</AVAILABILITY>
</PLANT>
<PLANT>
<COMMON>Marsh Marigold</COMMON>
<BOTANICAL>Caltha palustris</BOTANICAL>
<ZONE>4</ZONE>
<LIGHT>Mostly Sunny</LIGHT>
<PRICE>$6.81</PRICE>
<AVAILABILITY>051799</AVAILABILITY>
</PLANT>
<PLANT>
<COMMON>Cowslip</COMMON>
<BOTANICAL>Caltha palustris</BOTANICAL>
<ZONE>4</ZONE>
<LIGHT>Mostly Shady</LIGHT>
<PRICE>$9.90</PRICE>
<AVAILABILITY>030699</AVAILABILITY>
</PLANT>
<PLANT>
<COMMON>Dutchman's-Breeches</COMMON>
<BOTANICAL>Dicentra cucullaria</BOTANICAL>
<ZONE>3</ZONE>
<LIGHT>Mostly Shady</LIGHT>
<PRICE>$6.44</PRICE>
<AVAILABILITY>01

["Bloodroot Sanguinaria canadensis 4 Mostly Shady $2.44 031599 Columbine Aquilegia canadensis 3 Mostly Shady $9.37 030699 Marsh Marigold Caltha palustris 4 Mostly Sunny $6.81 051799 Cowslip Caltha palustris 4 Mostly Shady $9.90 030699 Dutchman's-Breeches Dicentra cucullaria 3 Mostly Shady $6.44 012099 Ginger, Wild Asarum canadense 3 Mostly Shady $9.03 041899 Hepatica Hepatica americana 4 Mostly Shady $4.45 012699 Liverleaf Hepatica americana 4 Mostly Shady $3.99 010299 Jack-In-The-Pulpit Arisaema triphyllum 4 Mostly Shady $3.23 020199 Mayapple Podophyllum peltatum 3 Mostly Shady $2.98 060599 Phlox, Woodland Phlox divaricata 3 Sun or Shade $2.80 012299 Phlox, Blue Phlox divaricata 3 Sun or Shade $5.59 021699 Spring-Beauty Claytonia Virginica 7 Mostly Shady $6.59 020199 Trillium Trillium grandiflorum 5 Sun or Shade $3.90 042999 Wake Robin Trillium grandiflorum 5 Sun or Shade $3.20 022199 Violet, Dog-Tooth Erythronium americanum 4 Shade $9.04 020199 Trout Lily Erythronium americanum 4 Sha

In [2]:
# vo=voyageai.Client(api_key=os.environ["VOYAGE_API_KEY"])
co=cohere.ClientV2(os.environ["COHERE_API_KEY"])


In [ ]:
#generate embeddings for text
def generate_embeddings(text):
    # embeddings=vo.embed(text,model="voyage-3-large", input_type="document")
    embeddings=co.embed(texts=[text], model="embed-english-v3.0",input_type="search_document",embedding_types=["float"])
    return embeddings.embeddings.float[0]


In [18]:
#generate embeddings for questions
def query_embeddings(query):
    q_embeddings=co.embed(texts=[query],model="embed-english-v3.0",input_type="search_query",embedding_types=["float"])
    return q_embeddings.embeddings.float[0]

In [12]:
#chuncking the texts
def chunk_documents(docs):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=200 , chunk_overlap=20)
    all_chunks=[]
    for doc in docs:
        chunks=text_splitter.split_text(doc)
        all_chunks.extend(chunks)
    # texts=text_splitter.split_documents(docs)
    return all_chunks


In [13]:
embeddings=[]
texts=chunk_documents(docs)
for text in texts:
    embedding=generate_embeddings(text)
    embeddings.append(embedding)

In [19]:
#testing the 
from numpy import dot
from numpy.linalg import norm
query="what does the mediterenian diet emphasize on?"
doc_embedds=embeddings
q_embeddings=query_embeddings(query)
similarity=dot(doc_embedds,q_embeddings)/(norm(doc_embedds)*norm(q_embeddings))
print(f"Similarity : {similarity}")

Similarity : [0.24762685 0.08435089 0.05709943 0.09577442 0.02234798 0.06167636]


In [25]:
def generate_embedded_doc(embeddings,data):
    finaldoc=[]
    for i, (embedding,data) in enumerate(zip(embeddings, data)):
        doc={
            "id":str(i),
            "values":embedding,
            "metadata":{
                'text':data,
                'chunk_index':i,
                'source':'custom text'
            }       
              }
        finaldoc.append(doc)
    return finaldoc

In [28]:
records_with_embeddings=generate_embedded_doc(embeddings,texts)

In [ ]:
pc=Pinecone(api_key=os.environ["PINECONE_API_KEY"])
index=pc.Index("medichatv1")

In [30]:
records_with_embeddings

[{'id': '0',
  'values': [0.060943604,
   0.017807007,
   -0.03378296,
   -0.055358887,
   0.035858154,
   -0.028686523,
   0.012031555,
   -0.0054969788,
   -0.023742676,
   0.025665283,
   0.038146973,
   0.01008606,
   -0.005455017,
   -0.04525757,
   0.015594482,
   -0.0057678223,
   0.017196655,
   -0.0050354004,
   0.005504608,
   -0.014572144,
   0.0061416626,
   0.0039787292,
   0.004142761,
   -0.021896362,
   -0.02217102,
   0.0073928833,
   -0.11419678,
   -0.017166138,
   -0.050354004,
   -0.048431396,
   -0.028442383,
   0.11077881,
   0.022964478,
   0.011146545,
   -0.014282227,
   0.005874634,
   -0.013519287,
   0.0074157715,
   -0.010124207,
   0.031677246,
   0.0058631897,
   0.0063667297,
   0.062561035,
   -0.07910156,
   -0.011238098,
   0.008094788,
   0.00094509125,
   -0.024337769,
   -0.008811951,
   -0.050933838,
   0.032989502,
   -0.035095215,
   0.05618286,
   0.0019626617,
   -0.044158936,
   -0.021347046,
   0.07836914,
   -0.06829834,
   0.043060303,
  

In [31]:
index.upsert(vectors=records_with_embeddings)

{'upserted_count': 6}

In [9]:
name="RodMan"
name

'RodMan'